# Índice

**Extractor de URLs**

**Extractor de contenido**

**Creación de ficheros .csv**

## Extractor de URLs

El fichero 1_Recipes_Extractor es el extractor de Información procedente de la web  www.food.com, de la que se origina el dataset inicial seleccionado (recipes.csv) de Kaggle (https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews?select=recipes.csv).

Para llevar a cabo esta extracción se ha generado un listado de URLs de una muestra de las recetas del fichero recipes.csv (en el que se encuentra el campo 'RecipeId' que nos permite construir la Url de cada una de las recetas), ordenado según reseñas y valoraciones. Basado en estas URL y empleando BeautifulSoup se ha generado el fichero 1_INGREDIENTS_DATA.csv que, junto con el fichero 1_RECIPES_DATA.csv (nuestra muestra de recetas del fichero inicial de Kaggle), alimentarán dos de las tablas principales del análisis.

In [1]:
import pandas as pd
import numpy as np
import random
import string
import requests
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

import warnings
warnings.filterwarnings('ignore')

In [4]:
df_inicial=pd.read_csv("../Data/1_Initial_data/recipes.csv")
df_resumen =df_inicial.sort_values(by=['ReviewCount','AggregatedRating'], ascending=False).head(200)

In [5]:
lista_ids= list(df_resumen["RecipeId"])
lista_urls_inicial=[]

for id_receta in lista_ids:
    lista_urls_inicial.append(f"https://www.food.com/recipe/-{id_receta}")

In [6]:
driver = webdriver.Chrome(ChromeDriverManager().install())
lista_url = []

for url in tqdm(lista_urls_inicial):
    driver.get(url)
    driver.implicitly_wait(30)
    sleep(10)
    lista_url.append((driver.current_url)+("?units=metric&scale=4"))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [37:00<00:00, 11.10s/it]


## Extractor de contenido

En base a la lista de Url generada en el punto anterior vamos a extraer en el siguiente código la información complementaria al dataset inicial para cada una de las recetas. El procedimiento tarda en torno a 25 minutos por norma general.

In [9]:
sopa_info=[]

for url in tqdm(lista_url):
    sopa_info.append(requests.get(url))
    sleep(7)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [25:37<00:00,  7.69s/it]


In [10]:
sopa_info_raw=[]

for sopa in tqdm(sopa_info):
    sopa_info_raw.append(BeautifulSoup(sopa.content, 'html.parser'))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:16<00:00, 12.20it/s]


## Creación de ficheros .csv

El objetivo de esta extracción es obtener la información relativa a las siguientes columnas:

1. `RecipeIngredientQuantities`: en el csv de Kaggle la información relativa a las cantidades viene en una unidad de medida que no es adecuada para el objetivo de análisis de este estudio, por lo que necesitaremos descargarla nuevamente para tener las cantidades traducidas al sistema métrico. Reemplazada por columna `cantidades`.   

2. `RecipeIngredientParts`: debido a la necesidad de conversión de unidades para la columna RecipeIngredientQuantities, descargaremos también de nuevo los ingredientes. Reemplazada por columna `ingredientes`.  

3. `RecipeServings y RecipeYield`: para el estudio que vamos a realizar queremos un número estándar de raciones, por lo que fijaremos un mismo número de servings para todas las cantidades y en lugar de RecipeYield crearemos una columna ligeramente diferente, en la que se especifica si la cantidad contenida en la nueva columna RecipeServings se refiere a raciones o unidades. Reemplazada por columnas `raciones y etiqueta_raciones`.

4. `RecipeId`: esta columna del csv original se utilizará como punto de partida para generar el listado de url del que nos serviremos para obtener toda la información nombrada en los puntos anteriores. Finalmente de la información de las url obtendremos de nuevo el RecipeId, que actuará como vínculo entre la nueva tabla de ingredientes y el set de datos original. Columna extra generada: `url`.

5. `RecipeInstructions`: La extracción de esta columna se hará en un csv aparte de los puntos anteriores puesto que, a diferencia del resto, esta alimentará al csv inicial. La columna que contiene las instrucciones en el set de datos de Kaggle contiene errores, por lo que dada su importancia la extraeremos y la anexionaremos al mismo. Columna reemplazo generada: `intrucciones`.

Las columnas a extraer del punto 1 al 4 conformarán la tabla de INGREDIENTES, mientras que el punto 5 se anexionará a la tabla RECETAS.

### Tabla ingredientes

In [13]:
diccionario_recetas_ini = {"url":[],
                       "cantidades":[],
                       "ingredientes": [],
                       "raciones": [],
                       "etiqueta_raciones": []
                      }

for receta in tqdm(sopa_info_raw):
    diccionario_recetas_ini["url"].append(receta.find_all("a", {"class": "action button button--growing svelte-1apyilh"}))
    diccionario_recetas_ini["cantidades"].append(receta.find_all("span", {"class": "ingredient-quantity svelte-1apyilh"}))
    diccionario_recetas_ini["ingredientes"].append(receta.find_all("span", {"class": "ingredient-text svelte-1apyilh"}))
    diccionario_recetas_ini["raciones"].append((receta.find_all("span", {"class": "value svelte-1o10zxc"}))[0].getText())
    diccionario_recetas_ini["etiqueta_raciones"].append((receta.find_all("dt", {"class": "facts__label svelte-1apyilh"}))[2].getText())
    


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 59.59it/s]


In [14]:
recetas_ini= pd.DataFrame(diccionario_recetas_ini)

In [15]:
diccionario_recetas_def = {"url":[],
                       "cantidades":[],
                       "ingredientes": [],
                       "raciones": [],
                       "etiqueta_raciones": []
                      }
for i in tqdm(range(0,(len(recetas_ini)))):  
    for registro in zip(recetas_ini["cantidades"][i],recetas_ini["ingredientes"][i]):
        
        diccionario_recetas_def["url"].append(recetas_ini["url"][i])
        diccionario_recetas_def["cantidades"].append(registro[0].getText())
        diccionario_recetas_def["ingredientes"].append(registro[1].getText())
        diccionario_recetas_def["raciones"].append(recetas_ini["raciones"][i])
        diccionario_recetas_def["etiqueta_raciones"].append(recetas_ini["etiqueta_raciones"][i])

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 3516.66it/s]


In [16]:
recetas_definitivo= pd.DataFrame(diccionario_recetas_def)

In [18]:
recetas_definitivo.to_csv('../Data/2_Extracted_data/1_INGREDIENTS_DATA.csv', header=True, index=False)
df_resumen.to_csv('../Data/2_Extracted_data/2_RECIPES_DATA.csv', header=True, index=False)

### Tabla recetas

In [35]:
diccionario_instrucciones = {"url":[],
                             "intrucciones":[],
                              }

for receta in tqdm(sopa_info_raw):
    diccionario_instrucciones["url"].append(receta.find_all("a", {"class": "action button button--growing svelte-1apyilh"}))
    diccionario_instrucciones["intrucciones"].append(receta.find_all("li", {"class": "direction svelte-1apyilh"}))
        

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 124.52it/s]


In [36]:
instrucciones_ini= pd.DataFrame(diccionario_instrucciones)

In [64]:
diccionario_instrucciones_def = {"url":[],
                       "intrucciones":[]
                                  }

for i in tqdm(range(0,(len(instrucciones_ini)))):  
    for registro in instrucciones_ini["intrucciones"][i]:
        
        diccionario_instrucciones_def["url"].append(instrucciones_ini["url"][i])
        diccionario_instrucciones_def["intrucciones"].append(registro.getText())
        #diccionario_recetas_def["intrucciones"].append(registro[0].getText())

100%|█████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 13357.23it/s]


In [65]:
instrucciones_def= pd.DataFrame(diccionario_instrucciones_def)

In [69]:
instrucciones_def.to_csv('../Data/2_Extracted_data/4_INSTRUCTIONS_DATA.csv', header=True, index=False)